In [65]:
import pandas as pd
import numpy as np
import os

In [11]:
# list CSVs in data dir with correspondant size
for file in os.listdir("../data/"):
    statinfo = os.stat("../data/" + str(file))
    print(file + "\t\t\t" + str(statinfo.st_size))

auctions.csv			1445883592
installs.csv			787837
desc.json			8451
clicks.csv			6735396
events.csv			748730801
target_competencia.csv			7016


In [12]:
# get just de first 100 rows to infer data types and the efficiently load whole file
df = pd.read_csv('../data/auctions.csv', engine='c', nrows=100)

In [15]:
df.head()

,auction_type_id,country,date,device_id,platform,ref_type_id,source_id
0,NaN,6333597102633388268,2019-03-11 14:18:33.290763,6059599345986491085,1,1,0
1,NaN,6333597102633388268,2019-03-11 14:18:34.440157,1115994996230693426,1,1,0
2,NaN,6333597102633388268,2019-03-11 14:18:35.862360,7463856250762200923,1,1,0
3,NaN,6333597102633388268,2019-03-11 14:18:36.167163,7829815350267792040,1,1,0
4,NaN,6333597102633388268,2019-03-11 14:18:37.728590,1448534231953777480,1,1,0


In [16]:
df.dtypes

auction_type_id    float64
country              int64
date                object
device_id            int64
platform             int64
ref_type_id          int64
source_id            int64
dtype: object

# Why is auction_type_id    float64?

In [47]:
df['auction_type_id'].nunique()

0

They are all NaN here. Let's try loading just this column.

In [29]:
full_auction_type_id_col = pd.read_csv('../data/auctions.csv', engine='c', usecols=['auction_type_id'])

In [35]:
full_auction_type_id_col['auction_type_id'].value_counts()

Series([], Name: auction_type_id, dtype: int64)

**It is now confirmed that this column is absolutely useless.**

# Country

In [45]:

df.country.nunique()

1

In [59]:
df.country.value_counts()

6333597102633388268    100
Name: country, dtype: int64

***
Looks like this will be converted into a few categories and that's it.<br>
The values are big, so no loading with int8 here.

# Device id

In [46]:
df.device_id.nunique()

90

In [61]:
df.device_id.value_counts().head()

2435544514359302887    3
1877625370646254788    3
777745369352088675     2
1825114268640785066    2
3936313574333788462    2
Name: device_id, dtype: int64

***
The values are big, so no loading with int8 here.

# Platform

In [48]:
df.platform.nunique()

2

In [49]:
df.platform.value_counts()

1    85
2    15
Name: platform, dtype: int64

***
This could be an int8.

# ref_type_id

In [50]:
df.ref_type_id.nunique()

2

In [51]:
df.ref_type_id.value_counts()

1    85
7    15
Name: ref_type_id, dtype: int64

***
This could be an int8.

# source_id

In [52]:
df.source_id.nunique()

1

In [53]:
df.source_id.value_counts()

0    100
Name: source_id, dtype: int64

In [54]:
# Let's see if in the whole file there is any other value for this field.

full_source_id_col = pd.read_csv('../data/auctions.csv', engine='c', usecols=['source_id'])

In [55]:
full_source_id_col['source_id'].value_counts()

0    13354597
1     4016739
5     1466494
2      582083
6      151406
Name: source_id, dtype: int64

***
This could be an int8.

## Lets try and load the whole thing using the above information

In [66]:
df = pd.read_csv('../data/auctions.csv', 
                 engine='c', 
                 usecols=['country', 'date', 'device_id', 'platform','ref_type_id', 'source_id'], 
                 dtype={'platform':np.int8, 'ref_type_id':np.int8, 'source_id':np.int8})

In [67]:
df.dtypes

country         int64
date           object
device_id       int64
platform         int8
ref_type_id      int8
source_id        int8
dtype: object

In [69]:
df.shape

(19571319, 6)

# Country

In [68]:
df.country.nunique()

1

There's no sense having a column containing only 1 country (that's 19571319 rows of 64 bit ints!! `¯\(°_o)/¯`)

# Date

In [70]:
df.date.nunique()

19570963

# device id

In [71]:
df.device_id.nunique()

206977

In [72]:
df.device_id.value_counts()

633139769114048761     27762
7202276635029175071    23055
7298861376107043945    18188
6302840482782120473    16400
5376802567578262905    16367
8963711959081981780    14362
795159065504552200     12275
6841615430383729819    12077
5384039226444052914    11632
3198179064438296471    11565
519658456951133298     10541
2435544514359302887    10274
8751190726444323885    10256
1835469468095030098    10121
1800317677967270083    10036
9027154272484048876     9958
7574657722181550308     9767
5051610571898428259     9642
320899491737875154      9388
8292793494956750501     9315
201759600719985554      9158
8021195662481548645     9128
7650408579713341483     9020
6392567838173461255     8994
1478017482044456084     8839
6203407103275384250     8823
68016455690089173       8809
8496718680483076445     8682
8491787427844905567     8513
4834359787191717922     8495
                       ...  
5751956180575781757        1
2048572451559832028        1
3744576664345883432        1
73670867056061

We'll have to leave this in int64 `¯\_(ツ)_/¯`

## Final loading command for this file

In [76]:
%%time
df = pd.read_csv('../data/auctions.csv', 
                 engine='c', 
                 usecols=['date', 'device_id', 'platform','ref_type_id', 'source_id'], 
                 dtype={'platform':np.int8, 'ref_type_id':np.int8, 'source_id':np.int8})

CPU times: user 26.1 s, sys: 2.06 s, total: 28.1 s
Wall time: 1min 40s


In [77]:
# play a sound when whole notebook finished executing
duration = 1  # seconds
freq = 1500  # Hz
os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

0